In [1]:
# Load packages
import glob
import os
import ipywidgets as widgets
from ipywidgets import Layout, interact, interactive

In [ ]:
print(os.getcwd())

In [ ]:
# Print test
print('test')

In [ ]:
all_files = Get_Trades()

In [ ]:
# Function to get data for BR_trades
def Trade_Select(trade):
    
    for i in range(0, len(all_files)):
        
        if (all_files[i]['Ticker'][0] == trade):
            
            return(all_files[i])  

In [ ]:
##### BRAZIL TRADES DROPDOWN MENU #####

# Create dropdown menu to selct trade
BR_trade_dropdown = interactive(Trade_Select,
                                trade = widgets.Combobox(
                                    options= ['DOLG21','DOLQ20'],
                                    description="Trades:")
                               )

In [ ]:
print(all_files)